<a href="https://colab.research.google.com/github/Ral151/GenAI-Hackathon/blob/main/MediBot_Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install "transformers>=4.43,<4.47" "datasets>=2.19" "accelerate>=0.32" \
                 "peft>=0.11" "trl>=0.9.6" "bitsandbytes>=0.43.2" \
                 einops sentencepiece tiktoken jsonschema

import torch, transformers, peft, trl, datasets, bitsandbytes
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)
print("TRL:", trl.__version__)

Torch: 2.8.0+cu126
Transformers: 4.46.3
PEFT: 0.17.1
TRL: 0.17.0


In [ ]:
!pip install --upgrade trl peft transformers datasets

  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
  Attempting uninstall: trl
    Found existing installation: trl 0.17.0
    Uninstalling trl-0.17.0:
      Successfully uninstalled trl-0.17.0


In [ ]:
!nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

# project paths  (do NOT change)
PROJ = "/content/drive/MyDrive/healthbot"
DATA_DIR = f"{PROJ}/data"
MODEL_OUT = f"{PROJ}/models/qwen2.5-1.5b-triage-admin-lora"

!mkdir -p "$DATA_DIR" "$MODEL_OUT"
!ls -R "$PROJ"

Fri Oct 24 05:20:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
MODEL_OUT = "/content/drive/MyDrive/healthbot/models/qwen2.5-1.5b-triage-admin-lora"

# 1. Define the quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# 2. Load the model with the new quantization_config
# Note: The torch_dtype is now specified inside the config
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

# You can now proceed with your model
print("Model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

RuntimeError: Failed to import transformers.models.qwen2.configuration_qwen2 because of the following error (look up to see its traceback):
cannot import name 'layer_type_validation' from 'transformers.configuration_utils' (/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py)

In [ ]:
from datasets import load_dataset
import os

dataset_path = os.path.join(DATA_DIR, "train.jsonl")
if os.path.exists(dataset_path):
    dataset = load_dataset("json", data_files=dataset_path)
    print("✅ Dataset loaded:", dataset)
else:
    dataset = None
    print("⚠️ No dataset found. Skipping fine-tuning step.")

In [ ]:
from peft import LoraConfig
import transformers
from trl import SFTTrainer, SFTConfig # Make sure to import SFTConfig

# Ensure your dataset is loaded and not None
if 'dataset' in locals() and dataset is not None:

    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

    messages_column = "messages"

    # --- THIS IS THE FIX ---
    # Create the SFTConfig object, but WITHOUT max_seq_length
    sft_config = SFTConfig(
        dataset_text_field=messages_column,
        max_length=1024,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=5,
        output_dir=MODEL_OUT,
        save_total_limit=2,
        save_strategy="steps",
        save_steps=50,
        report_to="none"
        # The SFTTrainer handles completion-only loss automatically
        # when a response_template is provided, so you often don't need a custom collator.
        # We can remove the collator for a simpler setup.
    )

    # Now, initialize the SFTTrainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset["train"],
        peft_config=peft_config,
        args=sft_config, # Pass the config objects
    )

    trainer.train()
    trainer.model.save_pretrained(MODEL_OUT)
    tokenizer.save_pretrained(MODEL_OUT)
    print(f"✅ Fine-tuned model saved to {MODEL_OUT}")

else:
    print("💡 Skipping fine-tuning because the dataset was not loaded.")

    """# This is the correct way
    from trl import SFTTrainer
    from trl.data import DataCollatorForCompletionOnlyLM

    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # the column in your JSON that contains the combined prompt+response text
    messages_column = "messages"

    collator = DataCollatorForCompletionOnlyLM(
        response_template="### Response:",
        tokenizer=tokenizer,
        mlm=False
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset["train"],
        peft_config=peft_config,
        dataset_text_field=messages_column,
        max_seq_length=1024,
        data_collator=collator,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=10,
            max_steps=100,   # adjust this for longer training
            learning_rate=2e-4,
            fp16=True,
            logging_steps=5,
            output_dir=MODEL_OUT,
            save_total_limit=2,
            save_strategy="steps",
            save_steps=50,
            report_to="none"
        ),
    )

    trainer.train()
    trainer.model.save_pretrained(MODEL_OUT)
    tokenizer.save_pretrained(MODEL_OUT)
    print(f"✅ Fine-tuned model saved to {MODEL_OUT}")

else:
    print("💡 Skipping fine-tuning because dataset not found.")"""

In [ ]:
# =========================================================================
# PASTE THIS ENTIRE BLOCK INTO A NEW CELL AT THE END OF YOUR NOTEBOOK
# =========================================================================

# 🧩 Install dependencies (safe to rerun)
print("Installing dependencies...")
!pip install -q git+https://github.com/openai/whisper.git soundfile gTTS
!sudo apt-get update -y && sudo apt-get install -y ffmpeg espeak

import os, time, tempfile, torch, whisper, psutil
import IPython.display as ipd
from base64 import b64decode
from google.colab import output
from transformers import AutoTokenizer, AutoModelForCausalLM
from gtts import gTTS

# ============================================================
# 1️⃣ Load your fine‑tuned Qwen model (using your smart loader logic)
# ============================================================

MODEL_PATH = "/content/drive/MyDrive/healthbot/models/qwen2.5-1.5b-triage-admin-lora"

print(f"🚀 Loading fine‑tuned model from {MODEL_PATH} …")

try:
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Model not found at {MODEL_PATH}. Make sure training completed and saved the model.")

    # Prepare tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

    # Check for GPU
    if torch.cuda.is_available():
        print("✅ GPU detected. Loading model in 4-bit quantized mode.")
        from transformers import BitsAndBytesConfig
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4",
        )
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            quantization_config=bnb_cfg,
            device_map="auto",
            trust_remote_code=True,
        )
        device = "cuda"
    else:
        print("💻 No GPU detected. Loading model on CPU (will be slower).")
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map="cpu", # Explicitly map to CPU
            trust_remote_code=True,
        )
        device = "cpu"

    print(f"✅ Model ready on {device.upper()}")

except Exception as e:
    print(f"FATAL ERROR: Could not load the model. Please check paths and GPU/RAM availability.")
    print(f"Error details: {e}")
    # We will stop here if the model fails to load
    model = None

# ============================================================
# 2️⃣ Define the AI's persona with a strong System Prompt
# ============================================================

SYSTEM_PROMPT = """You are a specialized AI assistant for the 'HealthPoint Clinic' administration desk. Your name is 'AdminHelper'.

Your ONLY purpose is to help users with administrative tasks, including:
- Booking, rescheduling, or canceling appointments.
- Providing information about clinic hours, location, and contact details.
- Answering questions about billing and accepted insurance.
- Guiding users on how to access their patient portal.

**CRITICAL RULES:**
1.  **DO NOT PROVIDE MEDICAL ADVICE.** You are not a doctor, nurse, or paramedic.
2.  If a user asks for medical advice, describes symptoms (like 'I have a headache'), or asks about treatment, you MUST politely decline and redirect them. Say: "I cannot provide medical advice. For any medical concerns, please consult with a healthcare professional or contact emergency services if this is an urgent matter."
3.  Do not answer questions outside of your administrative scope (e.g., weather, news). Politely state that you can only assist with clinic-related administrative tasks.
4.  Keep your answers concise and professional.
"""

# ============================================================
# 3️⃣ Define Audio, STT, and TTS Functions
# ============================================================

from google.colab import output
from base64 import b64decode

def record_audio(filename="voice_input.wav"):
    # This JavaScript is a self-contained, immediately-invoked function (IIFE).
    # It's a much more stable way to run JS in Colab.
    record_js_iife = """
    (async function() {
      try {
        const duration = 5000; // Recording duration in milliseconds
        const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
        const recorder = new MediaRecorder(stream);
        let data = [];

        recorder.ondataavailable = e => data.push(e.data);
        recorder.start();

        console.log("🎤 Recording... Speak now for 5 seconds!");

        // Wait for the specified duration
        await new Promise(resolve => setTimeout(resolve, duration));

        recorder.stop();

        // Important: Stop all audio tracks to release the microphone
        stream.getTracks().forEach(track => track.stop());

        console.log("⏳ Processing audio...");

        // Wait for the recorder to finish processing the data
        await new Promise(resolve => recorder.onstop = resolve);

        const blob = new Blob(data, { type: 'audio/wav' });
        const arrayBuffer = await blob.arrayBuffer();
        // Convert to base64
        const base64 = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)));
        return base64;
      } catch (e) {
        // If any error occurs in the JS, return it as a prefixed string
        console.error("JavaScript Audio Error:", e);
        return "JS_ERROR:" + e.name + ": " + e.message;
      }
    })()
    """
    try:
        print("⚡ Recording will start. Please allow microphone access if prompted.")
        # Execute the self-contained JavaScript
        audio_b64 = output.eval_js(record_js_iife)

        # Check if the JavaScript side sent back our specific error string
        if isinstance(audio_b64, str) and audio_b64.startswith("JS_ERROR:"):
            raise RuntimeError(f"JavaScript Error -> {audio_b64.replace('JS_ERROR:', '')}")

        if not audio_b64:
            raise RuntimeError("Audio capture returned empty. Is the microphone silent?")

        # Decode the base64 string and save as a .wav file
        with open(filename, "wb") as f:
            f.write(b64decode(audio_b64))
        print(f"✅ Audio successfully saved to {filename}")
        return filename

    except Exception as e:
        # This will now print the clear error from either Python or JavaScript
        print(f"🔴 Audio recording failed: {e}")
        return None

def transcribe(filepath):
    if not filepath: return "", "en"
    print("🧠 Transcribing...")
    result = stt_model.transcribe(filepath, fp16=torch.cuda.is_available())
    text = result["text"].strip()
    lang = result["language"]
    print(f"🗣️ You said: {text} (Language: {lang})")
    return text, lang

def speak(text, lang_code="en"):
    if not text.strip(): return
    try:
        tmp_path = os.path.join(tempfile.gettempdir(), "voice_reply.mp3")
        safe_lang = "zh" if lang_code == "zh" else "en"
        print(f"🔊 Speaking reply via gTTS ({safe_lang})...")
        tts_obj = gTTS(text=text, lang=safe_lang, slow=False)
        tts_obj.save(tmp_path)
        ipd.display(ipd.Audio(tmp_path, autoplay=True))
    except Exception as e:
        print(f"⚠️ Text-to-Speech error: {e}")

# ============================================================
# 4️⃣ Update `ask_model` to use conversation history and the system prompt
# ============================================================

def ask_model(user_prompt, messages_history):
    if not user_prompt:
        return "I'm sorry, I didn't catch that. Could you please repeat?", messages_history

    messages_history.append({"role": "user", "content": user_prompt})
    full_prompt = tokenizer.apply_chat_template(messages_history, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7, do_sample=True)
    reply_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    final_reply = reply_text[len(tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)):].strip()
    messages_history.append({"role": "assistant", "content": final_reply})

    print(f"🤖 AdminHelper replied: {final_reply}")
    return final_reply, messages_history

# ============================================================
# 5️⃣ Create the main continuous chat loop
# ============================================================

# =====================================================================
# CORRECTED: Button-Driven "Push to Talk" Chat
# This version uses the correct 'ipywidgets' library.
# =====================================================================

import ipywidgets as widgets  # <--- THIS IS THE CORRECTED LINE
import IPython.display

# 1. Initialize the conversation history.
messages = [{"role": "system", "content": SYSTEM_PROMPT}]

# 2. Define a function that runs ONE conversation turn.
def run_one_conversation_turn(b):
    IPython.display.clear_output(wait=True)
    display(button)
    print("----------------------------------------------------")

    try:
        audio_path = record_audio()
        if not audio_path:
            print("🔴 Recording failed or was silent. Please check mic permissions and try again.")
            return

        user_text, lang_code = transcribe(audio_path)
        if not user_text:
            print("🤔 I didn't hear anything clearly. Please try again.")
            return

        global messages
        model_reply, messages = ask_model(user_text, messages)
        speak(model_reply, lang_code)

    except Exception as e:
        print(f"An unexpected error occurred during the turn: {e}")

# 3. Create the button widget. This will now work correctly.
button = widgets.Button(description="🎤 Click to Speak")

# 4. Tell the button to call our function whenever it's clicked.
button.on_click(run_one_conversation_turn)

# 5. Display the initial welcome message and the button.
if model is not None:
    print("✅ AdminHelper is ready.")
    print("Click the '🎤 Click to Speak' button below to start.")
    print("====================================================")
    display(button)
else:
    print("🔴 Model not loaded. Cannot start the chat.")